In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator

In [3]:
#model.export(file = Path("/kaggle/working/model.h5"))

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, 
                                  zoom_range=0.2, width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
base = '../input/new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)'
train_set = train_datagen.flow_from_directory(base+'/train', target_size=(224, 224),
                                             batch_size = 32, class_mode='categorical')

valid_set = valid_datagen.flow_from_directory(base+'/valid', target_size=(224,224),
                                             batch_size=32, class_mode='categorical')

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [6]:
base_model = keras.applications.VGG16(weights='../input/vgg16-weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', input_shape=(224,224,3), include_top=False)
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
base_model.trainable = False
inputs = keras.Input(shape=(224,224,3))

x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(38)(x)

vgg16_model = keras.Model(inputs, outputs, name='pretrained_vgg16')
vgg16_model.summary()

Model: "pretrained_vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 38)                19494     
Total params: 14,734,182
Trainable params: 19,494
Non-trainable params: 14,714,688
_________________________________________________________________


In [8]:
vgg16_model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.CategoricalCrossentropy(from_logits=True),
                   metrics=[keras.metrics.CategoricalAccuracy()])

vgg16_history = vgg16_model.fit(train_set, steps_per_epoch=150, validation_data=valid_set,
                               epochs=20, validation_steps=100)

Epoch 1/20
150/150 [==============================] - 1974s 13s/step - loss: 3.6334 - categorical_accuracy: 0.0638 - val_loss: 3.1153 - val_categorical_accuracy: 0.3400
Epoch 2/20
150/150 [==============================] - 1956s 13s/step - loss: 3.1140 - categorical_accuracy: 0.2545 - val_loss: 2.7213 - val_categorical_accuracy: 0.4238
Epoch 3/20
150/150 [==============================] - 1955s 13s/step - loss: 2.7443 - categorical_accuracy: 0.3824 - val_loss: 2.4026 - val_categorical_accuracy: 0.5419
Epoch 4/20
150/150 [==============================] - 1939s 13s/step - loss: 2.4851 - categorical_accuracy: 0.4780 - val_loss: 2.1886 - val_categorical_accuracy: 0.6259
Epoch 5/20
 85/150 [================>.............] - ETA: 8:28 - loss: 2.2903 - categorical_accuracy: 0.5238

KeyboardInterrupt: 

In [ ]:
vgg16_result = vgg16_model.evaluate(valid_set)
print('Val loss : ', vgg16_result[0])
print('Val accuracy : ', vgg16_result[1])

In [ ]:
import seaborn as sns

plt.rc('figure', figsize=(10, 6))
sns.set(font_scale=1)

In [ ]:
train_acc = vgg16_history.history['categorical_accuracy']
val_acc = vgg16_history.history['val_categorical_accuracy']

epochs_list = list(range(1, 25 + 1))

plt.plot(epochs_list, train_acc, label='train acc')
plt.plot(epochs_list, val_acc, label='val acc')
plt.title("VGG-16's Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='best')

In [ ]:
train_loss = vgg16_history.history['loss']
val_loss = vgg16_history.history['val_loss']

plt.plot(epochs_list, train_loss, label='train loss')
plt.plot(epochs_list, val_loss, label='val loss')
plt.title("VGG-16's Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc='best')